# Собираем сиамскую сеть

In [1]:
import numpy as np
import sys 
import os
sys.path.append(os.path.abspath("./../models/"))
from vgg import gen_net

In [2]:
np.random.seed(12)
train_fn, val_fn, net = gen_net(False, 3)

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX 1070 (CNMeM is disabled, cuDNN 5110)


In [3]:
files = np.load('../datasets/data_set160_dim3.npz')
X_train, y_train, X_val, y_val = files['X_train'], files['y_train'], files['X_test'], files['y_test']

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# def transform(data, target):
#     datagen = ImageDataGenerator(zca_whitening=True)
#     datagen.fit(data)
#     _data, _target = [], []
#     for x, y in datagen.flow(data, target):
#         _data.append(x.reshape(3, 160, 160))
#         _target.append(y)
#     return np.array(_data), np.array(_target)
# X_train, y_train = transform(X_train, y_train)
# X_test, y_test = transform(X_test, y_test)
datagen_train = ImageDataGenerator(zca_whitening=True)
datagen_train.fit(X_train)
datagen_test = ImageDataGenerator(zca_whitening=True)
datagen_test.fit(X_val) 

In [4]:
fictional_target = np.zeros(batchsize)

def transform(data, target):
    return np.array([x for x, _ in datagen.flow(data, target)])

def iterate_minibatches(inputs, targets, batchs_per_epoch=100, batchsize=20, train=True, shuffle=False):
    assert len(inputs) == len(targets)

    left_indices = np.arange(len(inputs))
    
    if shuffle:
        np.random.shuffle(left_indices)
        
    for _ in range(batchs_per_epoch):
        full_lft_indxs = []
        full_pos_indxs = []
        full_neg_indxs = []
        
        for _ in range(batchsize):
            start_idx = np.random.randint(low=0, high=len(left_indices))
            full_lft_indxs.append(start_idx)
            
            pos_idxs = np.where(targets == targets[start_idx])[0]
            b_idxs = np.random.randint(low=0, high=len(pos_idxs), size=1)
            full_pos_indxs.append(pos_idxs[b_idxs[0]])
            
            neg_idxs = np.where(targets != targets[start_idx])[0]
            b_idxs = np.random.randint(low=0, high=len(neg_idxs), size=1)
            full_neg_indxs.append(neg_idxs[b_idxs[0]])

        full_lft_indxs = np.array(full_lft_indxs)
        full_pos_indxs = np.array(full_pos_indxs)
        full_neg_indxs = np.array(full_neg_indxs)
        
        inputs_left = transform(inputs[full_lft_indxs], fictional_target)
        inputs_positive = transform(inputs[full_pos_indxs], fictional_target)
        inputs_negative = transform(inputs[full_neg_indxs], fictional_target)
        
#         yield inputs[full_lft_indxs], inputs[full_pos_indxs], inputs[full_neg_indxs]
        yield inputs_left, inputs_positive, inputs_negative

In [5]:
num_epochs = 200
train_errors = []
val_errors = []
epoch = 0
batch_size = 40
batchs_per_epoch = 16

margin = 1.242

In [6]:
import time

for epoch in range(epoch, num_epochs):
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batchs_per_epoch=batchs_per_epoch,
                                     batchsize=batch_size, train=True, shuffle=True):
        inputs_left, inputs_positive, inputs_negative = batch
        err = train_fn(inputs_left, inputs_positive, inputs_negative, margin)
        train_err += err
        train_batches += 1
    print(err)

    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batchs_per_epoch=batchs_per_epoch,
                                     batchsize=batch_size, train=False, shuffle=True):
        inputs_left, inputs_positive, inputs_negative = batch
        err = val_fn(inputs_left, inputs_positive, inputs_negative, margin)
        val_err += err
        val_batches += 1

    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    train_errors.append(train_err / train_batches)
    val_errors.append(val_err / val_batches)

57.84502029418945
Epoch 1 of 200 took 53.326s
  training loss:		4668220617358.335938
  validation loss:		49.661549
49.60108184814453


KeyboardInterrupt: 

In [7]:
import pickle
import os

import lasagne as nn

def write_model_data(model, filename):
    """Pickels the parameters within a Lasagne model."""
    PARAM_EXTENSION = 'params'
    data = nn.layers.get_all_param_values(model)
    filename = os.path.join('./', filename)
    filename = '%s.%s' % (filename, PARAM_EXTENSION)
    with open(filename, 'w') as f:
        pickle.dump(data, f)\

In [8]:
write_model_data(net, '../model_params/vgg-dim3')

TypeError: write() argument must be str, not bytes

In [9]:
print epoch

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-9-f9e6dfd006fa>, line 1)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(train_errors, 'r')
plt.plot(val_errors, 'b')
plt.show()